# Webbskrapning

Webbskrapning (_eng. web scraping_) går ut på att automatisera sökandet av information från sidor på nätet. Här är några motiveringar till varför man kan göra det:

- När den data du behöver inte finns färdigt samlad
- I forskningssyfte för att skaffa information
- I kommersiellt syfte för att analysera kunders val på nätet och automatisera marknadsföring

Men är det lagligt? Jo, så länge man skrapar allmänt tillgänglig data:

- Användaren har gjort datan offentlig
- Det krävs inget konto för att nå datan
- Sidan är inte blockerad av en `robots.txt` fil

Som exempel har Yle en sådan fil med restriktioner, https://yle.fi/robots.txt. Vi tar och ser vad den innehåller. Ett lätt sätt att ladda sidor från internet i Python är genom att använda `requests.get()`:

In [1]:
import requests

restriktioner = requests.get('https://yle.fi/robots.txt')
print(restriktioner.text)

User-agent: *
Disallow: /ylefiroot-errors
Disallow: /cgi-bin
Disallow: /klaffi
Disallow: /java
Disallow: /player
Disallow: /errorsivut/
Disallow: /global/
Disallow: /haku/
Disallow: /eurovision/
Disallow: /embed/
Allow: /global/api/load.php
Allow: /cgi-bin/tekstitv/ttv.cgi/


#yle.fi/aihe, aka. FYND - contact drupal@yle.fi if needed
Disallow: /aihe/flag/
Disallow: /aihe/ajax_comments/
Disallow: /aihe/comment/reply/

# karttasovellus APla 9.7.2015
Disallow: /karttasovellus/


- `User-agent: *` i filen innebär att restriktionerna gäller alla sorters robotar
- `Disallow: /eurovision/` innebär att robotar inte får skrapa t.ex. https://yle.fi/eurovision/

<div class='alert alert-info'>Övning 1

Sök på olika sidor som du brukar använda ifall de har en `robots.txt` fil. Försök förstå vilka sidor de avråder robotar från att besöka. Om du vill veta mer om _The Robots Exclusion Protocol_ hittar man information här https://www.robotstxt.org.
</div>

## Dagens nyheter

Nu ska vi se om vi lyckas skrapa Svenska Yles framsida för att få dagens nyheter listade. Vi börjar med att importera `BeautifulSoup` som är ett paket i Python för att navigera HTML (HyperText Markup Language), d.v.s. språket som webbsidor skrivs i. Sedan söker vi sidan på nätet med `requests` paketet igen och omvandlar sidans innehåll med en HTML parser till en s.k. _soppa_ som `Beautifulsoup` kan hantera. En parser är något som förstår syntaxen (regler, grammatik, uppbyggnad) för ett språk som i vårt fall är HTML.

In [2]:
from bs4 import BeautifulSoup

yle = requests.get('https://svenska.yle.fi')
soppa = BeautifulSoup(yle.content, 'html.parser')

### Hitta HTML element

Man kan hitta information i sidan med `find()`, vi söker sidans titel som syns uppe i sidans flik i sökmotorn. HTML använder `<tagg></tagg>` för olika typer av innehåll, och genom att byta ut `tagg` till t.ex. `title` = titel, `header` = sidhuvud eller `footer` = sidfot får man olika sorters element på webbsidor. 

In [3]:
titel = soppa.find('title')
print(titel)

<title>Svenska Yle</title>


Sedan kollar vi på sidhuvudet som ofta har lite inledande information samt länkar för att navigera sidan. Själva länkarna finner vi genom HTML taggen `<a>`, man kan få alla på en gång som en lista genom att använda `find_all()`. Själva webaddressen finns i HTML under `href`. Man kan göra det lite snyggare då man skriver ut `BeautifulSoup` element genom att använda `prettify()`

In [4]:
sidhuvud = soppa.find('header')
sidhuvud_lankar = sidhuvud.find_all('a')
print(sidhuvud_lankar[1].prettify())

<a href="https://svenska.yle.fi" id="yle-header-main-link--svenska-yle" tabindex="0">
 Svenska Yle
</a>



Eftersom vi sökte länkarna som en lista kan man loopa igenom dem och deras texter! Man får texten för ett `Beautifulsoup` element med `text` funktionen och länkar fås med `get('href')`. Här skrivs allt ut som `f-strängar` för att få enhetliga mellanrum mellan text och länkar.

In [5]:
for lank in sidhuvud_lankar:
  print(f"{lank.text:20s}{lank.get('href')}")

svenska.yle.fi      https://svenska.yle.fi
Svenska Yle         https://svenska.yle.fi
Arenan              https://arenan.yle.fi/tv
Ämnen               https://svenska.yle.fi/lista/amnessidor
Svenska Yle         https://svenska.yle.fi
Arenan              https://arenan.yle.fi/tv
Inrikes             https://svenska.yle.fi/inrikes
Sport               https://svenska.yle.fi/sport
Utrikes             https://svenska.yle.fi/utrikes
Kultur              https://svenska.yle.fi/kultur
Familj              https://svenska.yle.fi/familj
Kunskap             https://svenska.yle.fi/vetamix
Huvudstadsregionen  https://svenska.yle.fi/huvudstadsregionen
Västnyland          https://svenska.yle.fi/vastnyland
Åboland             https://svenska.yle.fi/aboland
Österbotten         https://svenska.yle.fi/osterbotten
Östnyland           https://svenska.yle.fi/ostnyland
Kontakta oss        https://svenska.yle.fi/respons
Radio-guide         https://arenan.yle.fi/radio/guide
TV-guide            https://arenan.yle.

<div class='alert alert-info'>Övning 2

Se vad du hittar för länkar i sidfoten på https://svenska.yle.fi, tips: börja med att söka på `footer`. Skriv ut webbadresserna tillsammans med deras texter. Här är en grund för programmet:
    
```
import requests
from bs4 import BeautifulSoup

yle = requests.get('https://svenska.yle.fi')
soppa = BeautifulSoup(yle.content, 'html.parser')
```
</div>

### Spara innehåll i listor

Nyheterna som vi är intresserade av att hitta finns i huvudinnehållet i sidan innanför `<body>...</body>` så vi börjar med att söka på det.

In [6]:
innehall = soppa.find('body')

Rubriker i HTML finns innanför `<h1>`, `<h2>`, o.s.v. beroende på vilken nivå/storlekt rubriken har. Vi ser om vi hittar något som skulle likna en nyhet.

In [7]:
print(innehall.find('h1').prettify())
print(innehall.find('h2').prettify())

<h1 class="visually-hidden">
 Innehåll från svenska.yle.fi
</h1>

<h2 class="visually-hidden focusable">
 Huvudinnehåll
</h2>


Det verkar som att `<h1>` och `<h2>` inte fungerar so nyhetsubriker på sidan, men se däremot på `<h3>`:

In [8]:
print(innehall.find('h3').prettify())

<h3 class="js-ydd-card-title flex txt-16">
 Ted &amp; Kaj: Din mamma var en sköldpadda
</h3>



Nu vet vi hur vi hittar nyhternas rubriker, men vad är de utan länkar till själva nyheten? Dem kan vi hitta genom att igen söka på alla `<a>` taggar.

In [9]:
innehall_lankar = innehall.find_all('a')

Vi vill hitta de länkar som är associerade till en nyhetsrubrik, så vi söker efter rubrik taggen `<h3>` i varje länk. Nu för att se att det fungerar avslutas loopen med `break` efter att vi hittat den första rubriken. För att få bort onödiga mellanrum som finns kring en del rubriker använder vi `strip()` funktionen.

In [10]:
for lank in innehall_lankar:
  rubrik = lank.find('h3')
  if rubrik:
    print('Rubrik:', rubrik.text.strip())
    print('Länk:', lank.get('href'))
    break

Rubrik: Ted & Kaj: Din mamma var en sköldpadda
Länk: https://arenan.yle.fi/1-50576822


Det ser ju bra ut, då gör vi lika för alla länkar i innehållet och sparar resultatet i två listor, `nyhetsrubriker` och `nyhetslankar`.

In [11]:
nyhetsrubriker = []
nyhetslankar = []

for lank in innehall_lankar:
  rubrik = lank.find('h3')
  if rubrik:
    nyhetsrubriker.append(rubrik.text.strip())
    nyhetslankar.append(lank.get('href'))

<div class='alert alert-info'>Övning 3

Det finns flera hundra robotar listade på https://www.robotstxt.org/db.html, spara robotarnas namn och webaddresser i varsin lista. Tips: `find_all()` och `<li>` (_eng. list item_) som är en tagg för listade element.
    
Man kan se hela sidans innehåll så här:

```
import requests
from bs4 import BeautifulSoup

robotar = requests.get('https://www.robotstxt.org/db.html')
soppa = BeautifulSoup(robotar.content, 'html.parser')
    
print(soppa.prettify())
```    

Ett annat mycket bra sätt att se sidans uppbyggnad är att besöka sidan, högerklicka på den och välja `Visa sidkälla`, <view-source:https://www.robotstxt.org/db.html>.

</div>

### Skapa en tabell

Det skulle vara snyggt att visa nyheterna i tabellform så vi använder oss av paketet `pandas` som är bra på just det. Man anger kolumnernas namn som nycklar och listorna som värden i ett Python dictionary.

In [12]:
import pandas as pd

nyheter = pd.DataFrame({'Rubrik': nyhetsrubriker, 'Länk': nyhetslankar})

Rubrikerna och länkarna kan vara väldigt långa så för att undvika att de kapas av automatiskt sätter vi `max_colwidth` till `None`, det gör att det inte finns nån maximal bredd för tabellens innehåll. Men hur vet man att man ska ändra på just någon sån parameter? Det lär man sig genom att söka på nätet, oftast landar man på [Stack Overflow](https://stackoverflow.com/questions/25351968/how-to-display-full-non-truncated-dataframe-information-in-html-when-convertin) då det handlar om programmeringstips. Det är bra att veta att alla programmerare konstant söker information på internet eftersom det är omöjligt att veta och minnas hur man ska koda olika saker.

Till slut skriver vi ut topp 10 nyheterna!

In [13]:
pd.set_option('display.max_colwidth', None)
nyheter.head(10)

,Rubrik,Länk
0,Ted & Kaj: Din mamma var en sköldpadda,https://arenan.yle.fi/1-50576822
1,Senaste nytt om coronaviruset,https://svenska.yle.fi/coronavirus
2,"Jobb som bärplockare är ökänt för dåliga arbetsförhållanden men så ska det inte vara: ""Det är viktigt att alla behandlas lika oavsett varifrån de kommer""",https://svenska.yle.fi/artikel/2020/07/01/jobb-som-barplockare-ar-okant-for-daliga-arbetsforhallanden-men-sa-ska-det-inte
3,Preliminära uppgifter: Förlängt Putinstyre i sikte,https://svenska.yle.fi/artikel/2020/07/01/forlangt-putinstyre-i-sikte-enligt-preliminara-uppgifter
4,Kommentar: Så länge ryssarna sväljer maktfullkomlighet och valfusk sitter Putin stadigt kvar,https://svenska.yle.fi/artikel/2020/07/01/kommentar-sa-lange-ryssarna-svaljer-maktfullkomlighet-och-valfusk-sitter-putin
5,Att vara kvinna och jägare är att våga ta plats och bevisa sin duglighet,https://svenska.yle.fi/artikel/2020/07/01/att-vara-kvinna-och-jagare-ar-att-vaga-ta-plats-och-bevisa-sin-duglighet
6,"Trumps säkerhetsrådgivare: Uppgifter om ryska skottpengar så ""vaga"" att presidenten inte informerades",https://svenska.yle.fi/artikel/2020/07/01/trumps-sakerhetsradgivare-uppgifter-om-ryska-skottpengar-sa-vaga-att-presidenten
7,Tusentals Hongkongbor ute på gatorna i protest mot ny inskränkande lag - polisen grep människor med hänvisning till lagen,https://svenska.yle.fi/artikel/2020/07/01/tusentals-hongkongbor-ute-pa-gatorna-i-protest-mot-ny-inskrankande-lag-polisen
8,"Coronarestriktionerna sätter sin prägel på sommarens evenemang, folksamlingar på över 500 personer tillåtna utomhus igen – läs det senaste om coronaviruset här",https://svenska.yle.fi/coronavirus
9,Stor blunder av Tim Krul när Norwich åkte på ny förlust – Teemu Pukki fick bara spela en halvlek,https://svenska.yle.fi/artikel/2020/07/01/stor-blunder-av-tim-krul-nar-norwich-akte-pa-ny-forlust-teemu-pukki-fick-bara


Man kan spara nyheterna t.ex. som en `csv` fil med `pandas` funktionen `to_csv()` ifall man tänker använda datan man skrapar från nätet till något mer. Man kan t.ex. räkna statistik om man skrapar numror, eller se hur nyhets rubriker förändras olika dagar om man skrapar nyheter. När man skriver tabllen till en fil behöver man inte bry sig om `max_colwidth`, all data kommer nog med ändå. 

In [14]:
nyheter.to_csv('nyheter.csv')

Hela programmet blir att se ut så här:

```
import requests
from bs4 import BeautifulSoup
import pandas as pd

yle = requests.get('https://svenska.yle.fi')
soppa = BeautifulSoup(yle.content, 'html.parser')

innehall = soppa.find('body')
innehall_lankar = innehall.find_all('a')

nyhetsrubriker = []
nyhetslankar = []
for lank in innehall_lankar:
  rubrik = lank.find('h3')
  if rubrik:
    nyhetsrubriker.append(rubrik.text.strip())
    nyhetslankar.append(lank.get('href'))

nyheter = pd.DataFrame({'Rubrik': nyhetsrubriker, 'Länk': nyhetslankar})
print(nyheter.head(10))

nyheter.to_csv('yle_nyheter.csv')
```

I nästa kapitel visualiserar vi nyhtetsrubrikerna som ordmoln!